In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,061 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [991 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,252 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,219 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [938 kB]
Get:13 http://secu

In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

#Show the home data
home_sales_df.show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [7]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView('home_sales_data')

In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
four_bedroom = """SELECT YEAR(date) AS YEAR, ROUND(AVG(price), 2) AS AVG_PRICE FROM home_sales_data WHERE bedrooms = 4 GROUP BY YEAR ORDER BY YEAR ASC"""

spark.sql(four_bedroom).show()



+----+---------+
|YEAR|AVG_PRICE|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [9]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
three_bed_bath = """SELECT YEAR(date_built) AS YEAR, ROUND (AVG(price), 2) AS AVG_PRICE FROM home_sales_data WHERE bedrooms = 3 and bathrooms = 4 GROUP BY YEAR ORDER BY YEAR ASC"""

spark.sql(three_bed_bath).show()


+----+---------+
|YEAR|AVG_PRICE|
+----+---------+
|2010|678841.43|
|2011|742461.77|
|2012|750640.47|
|2013|722022.17|
|2014|735647.67|
|2015|739785.77|
|2016|777294.89|
|2017|704210.47|
+----+---------+



In [10]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

four_desires = """SELECT YEAR(date_built) AS YEAR, ROUND (AVG(price), 2) AS AVG_PRICE FROM home_sales_data WHERE bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living > 2000 GROUP BY YEAR ORDER BY YEAR ASC"""

spark.sql(four_desires).show()

+----+---------+
|YEAR|AVG_PRICE|
+----+---------+
|2010|285010.22|
|2011|276553.81|
|2012|307539.97|
|2013|303676.79|
|2014|297619.46|
|2015|297609.97|
|2016| 293965.1|
|2017|280527.62|
+----+---------+



In [11]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

view_rating = """SELECT view, ROUND (AVG(price), 2) AS AVG_PRICE FROM home_sales_data WHERE price >=  350000 GROUP BY view ORDER BY view ASC"""

spark.sql(view_rating).show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|AVG_PRICE|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 0.00011372566223144531 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales_data")

DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_data')

True

In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

cache_view = """SELECT view, ROUND (AVG(price), 2) AS AVG_PRICE FROM home_sales_data WHERE price >=  350000 GROUP BY view ORDER BY view ASC"""

spark.sql(cache_view).show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|AVG_PRICE|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 8.535385131835938e-05 seconds ---


In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.parquet('parquet_home_sales', mode='overwrite')



In [16]:
# 11. Read the parquet formatted data.
p_home_sales_df=spark.read.parquet('parquet_home_sales')

In [17]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df.createOrReplaceTempView('parquet_home_sales')

In [20]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""SELECT view, ROUND (AVG(price), 2) AS AVG_PRICE FROM parquet_home_sales WHERE price >=  350000 GROUP BY view ORDER BY view ASC""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|AVG_PRICE|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 0.9525845050811768 seconds ---


In [22]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table parquet_home_sales")

DataFrame[]

In [25]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("parquet_home_sales")


False